# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)
df.show(3)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 3 rows



In [4]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



### Change Cruise_line to categorical variable

In [5]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [6]:
from pyspark.ml.feature import StringIndexer

In [7]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_class")

In [8]:
indexed = indexer.fit(df).transform(df)
indexed.show(3)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_class|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
only showing top 3 rows



In [9]:
indexed.groupBy('Cruise_line_class').count().show()

+-----------------+-----+
|Cruise_line_class|count|
+-----------------+-----+
|              8.0|    6|
|              0.0|   23|
|              7.0|    8|
|             18.0|    2|
|              1.0|   22|
|              4.0|   13|
|             11.0|    4|
|             14.0|    3|
|              3.0|   14|
|             19.0|    1|
|              2.0|   17|
|             17.0|    2|
|             10.0|    5|
|             13.0|    3|
|              6.0|   10|
|              5.0|   11|
|             15.0|    3|
|              9.0|    6|
|             16.0|    2|
|             12.0|    3|
+-----------------+-----+



### Assemble features so that the data is acceptable by the ML LR model

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_class']

In [12]:
assembler = VectorAssembler(inputCols=['Age',
                                       'Tonnage',
                                       'passengers',
                                       'length',
                                       'cabins',
                                       'passenger_density',
                                       'Cruise_line_class'], outputCol='features')
                                      
output = assembler.transform(indexed)

In [13]:
output.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_class',
 'features']

In [14]:
data = output.select('features', 'crew')
data.show(3)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
+--------------------+----+
only showing top 3 rows



### Modeling

In [15]:
train, test = data.randomSplit([0.7, 0.3], seed=123)

In [16]:
from pyspark.ml.regression import LinearRegression

In [17]:
lr = LinearRegression(labelCol='crew')

In [18]:
model = lr.fit(train)

In [19]:
print("Coefficients: {}\n\nIntercept: {}".format(model.coefficients, model.intercept))

Coefficients: [-0.0091272681129,0.0135617936549,-0.170214149016,0.35543505843,0.895384744527,0.00112769461599,0.0401579862953]

Intercept: -0.93999394071


In [20]:
result = model.evaluate(test)

In [21]:
print("RMSE: {}".format(result.rootMeanSquaredError))
print("MSE: {}".format(result.meanSquaredError))
print("R2: {}".format(result.r2))

RMSE: 0.823233078942
MSE: 0.677712702265
R2: 0.93890653997


In [22]:
result.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|-1.2436453515612982|
|0.39555679536728405|
| 0.7793250332026043|
|-1.1145399358287413|
| 0.5293319265632466|
+-------------------+
only showing top 5 rows



### Find which feature(s) are relevant

In [23]:
from pyspark.sql.functions import corr

In [24]:
inputCols = ['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'Cruise_line_class']

for col in inputCols:
    indexed.select(corr('crew', col)).show()

+-------------------+
|    corr(crew, Age)|
+-------------------+
|-0.5306565039638852|
+-------------------+

+-------------------+
|corr(crew, Tonnage)|
+-------------------+
|  0.927568811544939|
+-------------------+

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

+------------------+
|corr(crew, length)|
+------------------+
|0.8958566271016579|
+------------------+

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

+-----------------------------+
|corr(crew, passenger_density)|
+-----------------------------+
|         -0.15550928421699717|
+-----------------------------+

+-----------------------------+
|corr(crew, Cruise_line_class)|
+-----------------------------+
|         -0.48332562728617057|
+-----------------------------+



### Print model summary with predictions

In [25]:
model.summary.predictions.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[4.0,220.0,54.0,1...| 21.0|21.238100315582148|
|[5.0,133.5,39.59,...|13.13|12.947576235178119|
|[5.0,160.0,36.34,...| 13.6|15.269905580007661|
|[6.0,30.276999999...| 3.55| 4.215079452224267|
|[6.0,30.276999999...| 3.55| 4.215079452224267|
|[6.0,93.0,23.94,9...|11.09|10.543709119924479|
|[6.0,112.0,38.0,9...| 10.9|11.101007342739747|
|[6.0,113.0,37.82,...| 12.0|11.535565383504478|
|[6.0,158.0,43.7,1...| 13.6|13.865994782227762|
|[7.0,89.6,25.5,9....| 9.87|11.023410591174603|
|[8.0,91.0,22.44,9...| 11.0|10.084030742364561|
|[8.0,110.0,29.74,...| 11.6|12.193046391616646|
|[9.0,59.058,17.0,...|  7.4| 7.528174361998792|
|[9.0,81.0,21.44,9...| 10.0| 9.502292174659724|
|[9.0,85.0,19.68,9...| 8.69| 9.324829343892395|
|[9.0,88.5,21.24,9...| 10.3| 9.581705480132728|
|[9.0,105.0,27.2,8...|10.68|11.321132226269873|
|[9.0,110.0,29.74,...| 11.6|12.187473474